# L5: Content Creation at Scale

# Import libs

In [1]:
# Warning control
import warnings
import os
import yaml
import textwrap

from pydantic import BaseModel, Field
from typing import List
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

from IPython.display import display, Markdown


load_dotenv()  # take environment variables from .env
warnings.filterwarnings("ignore")

# os.environ["OPENAI_MODEL_NAME"] = "gpt-4o"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"
groq_llm = "groq/gemma2-9b-it"


# Loading Tasks and Agents YAML files

In [2]:
# Define file paths for YAML configurations
files = {"agents": "config/L5_agents.yaml", "tasks": "config/L5_tasks.yaml"}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, "r") as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs["agents"]
tasks_config = configs["tasks"]

In [3]:
configs.keys()

dict_keys(['agents', 'tasks'])

# Creating Structured Output

In [4]:
class SocialMediaPost(BaseModel):
    platform: str = Field(
        ...,
        description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).",
    )
    content: str = Field(
        ...,
        description="The content of the social media post, including any hashtags or mentions.",
    )


class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(
        ..., description="A list of social media posts related to the article."
    )

# Creating Crew, Agents, and Tasks

In [5]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config["market_news_monitor_agent"],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config["data_analyst_agent"],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config["content_creator_agent"],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config["quality_assurance_agent"],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config["monitor_financial_news"], agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config["analyze_market_data"], agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config["create_content"],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task],
)

quality_assurance_task = Task(
    config=tasks_config["quality_assurance"],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput,
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent,
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task,
    ],
    verbose=True,
)

# Kicking off the Crew

In [6]:
result = content_creation_crew.kickoff(
    inputs={"subject": "How AI agents have evolved since the launch of ChatGPT"}
)

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on How AI agents have evolved since the launch of ChatGPT. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to How AI agents have evolved since the launch of ChatGPT.



# Agent: Lead Market Analyst
## Thought: Thought: To get a clear understanding of how AI agents have evolved since ChatGPT's launch, I need to search for recent news articles and analyses on the topic.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Financial News AI agents developments since ChatGPT launch\"}"
## Tool Output: 

Search results: Title: After ChatGPT and the rise of chatb

Inserting batches in chromadb: 100%|██████████| 1/1 [00:01<00:00,  1.22s/it]




# Agent: Chief Data Strategist
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"impact of AI agents on investment strategies\", \"website\": \"https://www.investopedia.com\"}"
## Tool Output: 
Relevant Content:
Your Roth IRA Updated Nov 08, 2024 S&P 500 Surpasses 6K for First Time After Climbing 1K Points in Record Time Nov 08, 2024 Consumer Sentiment Revved Up on the Eve of the Election Nov 08, 2024 ORIGINAL RESEARCH Best Credit Repair Companies for 2024 We evaluated 17 companies on 69 factors to find the best credit repair companies to help fix your credit and boost your score. Latest Podcast Episodes The Investopedia Express Why the Federal Budget Deficit Could Trigger the Next Financial Crisis Episode  212 Nov 04, 2024 What Actually Happened to Investors Who Joined the Market During Meme Stock Mania? Episode  211 Oct 28, 2024 How to Really Get Better With Our Money Episode  210 Oct 21, 2024 Investopedia Special Features Our Special Features and Br

# Social Content

In [7]:
posts = result.pydantic.dict()["social_media_posts"]
for post in posts:
    platform = post["platform"]
    content = post["content"]
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print("-" * 50)

Twitter
🚀 The era of AI agents has arrived! Since
ChatGPT's launch, we've seen a transformation in
AI capabilities. Discover how these autonomous
agents are redefining industries and creating new
opportunities for businesses! #AI #ChatGPT
#Innovation
--------------------------------------------------
LinkedIn
AI technology is evolving! Post-launch of ChatGPT,
AI agents have emerged, capable of autonomously
performing tasks and adapting to environments.
From Microsoft’s "virtual employees" to JPMorgan’s
AI assistant, these advancements are set to
revolutionize sectors including finance and
accounting. Are you ready to embrace the AI agent
revolution? 🌐 #AIEvolution #BusinessTransformation
--------------------------------------------------
Instagram
🤖✨ AI agents have taken center stage since the
launch of ChatGPT! From automating complex tasks
to enhancing customer experiences, the evolution
is reshaping how businesses operate. Swipe to
explore key developments and trends in AI! #AI
#Cha

# Blog Post

In [8]:
display(Markdown(result.pydantic.dict()["article"]))

# Beyond Chatbots: The Evolution of AI Agents and Their Impact on Industries

As we stand on the brink of a new technological revolution, the advent of AI agents is reshaping the landscape of artificial intelligence, moving beyond the capabilities of earlier chatbots like ChatGPT. Since the launch of OpenAI's groundbreaking model in 2022, AI agents have emerged as autonomous entities capable of performing complex tasks, learning from interactions, and adapting to dynamic environments.

## The Shift from Chatbots to Autonomous AI Agents

The release of ChatGPT sparked a surge of interest in generative AI, but the conversation quickly pivoted towards more advanced applications—AI agents. Here are the key developments that mark this evolution:

1. **Microsoft's Bold Initiative**: Microsoft has taken a leading role in this domain, integrating AI agents into its Office suite and Bing search engine. Their agents, branded as "virtual employees," can handle client queries, schedule meetings, and summarize documents, redefining productivity in the workplace (Source: [The Guardian](https://www.theguardian.com)).  
2. **Influx of Investments**: Investors are recognizing the potential for disruption in the AI sector. Recently, OpenAI secured a massive $6.6 billion in funding, pushing its valuation to an astounding $157 billion (Source: [Reuters](https://www.reuters.com)).  
3. **Real-World Applications**: Major corporations like JPMorgan Chase are implementing AI assistants powered by OpenAI technology to streamline operations and enhance productivity (Source: [CNBC](https://www.cnbc.com)).  
4. **Revolutionizing Industries**: Various sectors, including finance and accounting, are set to benefit from AI agents' ability to automate complex processes, leading to enhanced efficiency and significant cost savings (Source: [Citywire](https://www.citywire.com)).

## Understanding AI Agents

Unlike traditional chatbots, AI agents are designed to:

- **Learn and Adapt**: They can learn from previous interactions, improving their decision-making capabilities.
- **Perform Complex Tasks**: AI agents can undertake a wider range of activities, from analyzing data to managing workflows, all while operating independently.
- **Enhance Customer Experience**: By automating tasks, businesses can streamline operations and provide tailored services to customers.

The distinct capabilities of AI agents offer transformative potential for various industries, highlighting their relevance in today's fast-paced, technology-driven world. Moreover, their ability to continuously improve ensures that businesses will be able to grow and evolve alongside their AI counterparts.

## Key Takeaways for Investors and Businesses

- **Investing in AI**: The rising investments in AI agents signify their immense growth potential and the disruptive capabilities they bring.
- **Adapting to Change**: Businesses need to shift their focus from simple chatbot solutions to smarter, autonomous agents to stay competitive.
- **Seizing New Opportunities**: Harnessing the power of AI agents allows companies to automate processes, improve customer interactions, and gain a competitive edge.

As industries increasingly adopt these autonomous agents, they can expect not only operational improvements but also a radical shift in how they interact with customers and deliver value.

## Conclusion

The evolution of AI agents since the launch of ChatGPT represents a significant leap in technology. As these agents continue to develop, they offer exciting possibilities for automation and intelligent systems that can transform industries. Staying informed and adapting to these changes will be critical for investors and businesses alike in navigating the future of work.